## Investigating Data Distribution

In this notebook, we attempt to quantify how seriously imbalanced the data distribution is

In [3]:
import os
os.chdir('..')

In [22]:
import pandas as pd
from ssoc_autocoder import train
pd.options.display.max_rows = 999

In [5]:
data = pd.read_csv('Data/Processed/Training/train-aws/train_full.csv')

In [9]:
SSOC_2020 = pd.read_csv('Data/Processed/Training/train-aws/SSOC_2020.csv')

In [11]:
colnames = {
    'SSOC': 'SSOC 2020',
    'job_description': 'Cleaned_Description'
}

In [12]:
encoding = train.generate_encoding(SSOC_2020)
encoded_data = train.encode_dataset(data, encoding, colnames)

In [16]:
data['SSOC 2020'].astype('str').str.slice(0, 1).value_counts()

2    15882
3     9306
1     7807
4     4039
5     2054
8     1643
9     1103
7     1001
6        7
Name: SSOC 2020, dtype: int64

In [25]:
data['SSOC 2020'].astype('str').str.slice(0, 5).value_counts().sort_values(ascending = False).head(20)

21499    3720
33499    2621
13499    1899
25121    1654
41109    1512
12211    1490
24212    1273
33299    1001
25190     898
43112     841
14121     810
25111     671
94101     631
21522     612
83329     562
33492     543
42245     449
11202     444
51312     417
83441     413
Name: SSOC 2020, dtype: int64

In [66]:
selected = data[data['SSOC 2020'] == 12211].sample(1)
print(selected.index[0])
print(selected.Cleaned_Description.values)

22952
['Using predictive and prescriptive analytics to forecast demand and design appropriate revenue models. Maintaining of Revenue Management standards specific review and measurement of company revenue performance against established goals. Overall development and implementation of strategy to improve revenue per acquired room for the properties across countries. Monitoring group and transient business thresholds within the forecast to ensure measurable improvements for demand and availability. Working with different departments like marketing, corporate sales and business development and control the impact that they create on revenue by setting individual targets for them. Working with external partners such OTAs & other tech providers to ensure the best interest of the company.']
